In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [27]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [14]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [15]:
dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [17]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid= {'solver':['newton-cg','lbfgs', 'liblinear', 'sag', 'saga'], 
             'penalty':['l2'], 
             'random_state':[0]}
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'], 'random_state': [0],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='f1_weighted', verbose=3)

In [25]:
re=grid.cv_results_

grid_predictions=grid.predict(X_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,grid_predictions)

from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,grid_predictions)

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

In [26]:
from sklearn.metrics import f1_score
f1_micro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_micro value for best parameter{}:".format(grid.best_params_),f1_micro)

The f1_micro value for best parameter{'penalty': 'l2', 'random_state': 0, 'solver': 'newton-cg'}: 0.8906190214115367


In [38]:
re

{'mean_fit_time': array([0.01978817, 0.01519084, 0.00279856, 0.00539708, 0.00439668]),
 'std_fit_time': array([0.00782861, 0.00556148, 0.00040007, 0.00174297, 0.00080013]),
 'mean_score_time': array([0.0021976 , 0.00259795, 0.00219893, 0.00219693, 0.0025991 ]),
 'std_score_time': array([0.00039919, 0.00079925, 0.00074756, 0.00039845, 0.0008009 ]),
 'param_penalty': masked_array(data=['l2', 'l2', 'l2', 'l2', 'l2'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_random_state': masked_array(data=[0, 0, 0, 0, 0],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_solver': masked_array(data=['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'penalty': 'l2', 'random_state': 0, 'solver': 'newton-cg'},
  {'penalty': 'l2', 'random_state': 0, '

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html

In [39]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9481321395492436

In [34]:
table=pd.DataFrame.from_dict(re)

In [37]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_random_state,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019788,0.007829,0.002198,0.000399,l2,0,newton-cg,"{'penalty': 'l2', 'random_state': 0, 'solver':...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
1,0.015191,0.005561,0.002598,0.000799,l2,0,lbfgs,"{'penalty': 'l2', 'random_state': 0, 'solver':...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
2,0.002799,0.000400,0.002199,0.000748,l2,0,liblinear,"{'penalty': 'l2', 'random_state': 0, 'solver':...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
3,0.005397,0.001743,0.002197,0.000398,l2,0,sag,"{'penalty': 'l2', 'random_state': 0, 'solver':...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
4,0.004397,0.000800,0.002599,0.000801,l2,0,saga,"{'penalty': 'l2', 'random_state': 0, 'solver':...",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1


In [29]:
print("The Confusion Matrix:\n ",cm)

The Confusion Matrix:
  [[74  5]
 [ 8 33]]


In [36]:
print("The Classification Report:\n ",clf_report)

The Classification Report:
                precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120

